<a href="https://colab.research.google.com/github/amiranissian/ai-ml-principles-exercises/blob/main/ML-training/logging-trainings-with-WandB/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install dependencies and import packages
First we need to install the libraries we will be using. We will use `numpy` for generic matrix operations and `tensorflow` for deep learning operations such as convolutions, pooling and training (backpropagation).

In [2]:
import sys
!{sys.executable} -m pip install numpy tensorflow
from IPython.display import clear_output
clear_output()
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

## Create a neural network
Next we define a function that can be used to build a neural network. The neural network is a simple CNN (convolutional neural network) used for classification. The structure of the network is not important for this exercise, you can instead see it as a black box that can be trained to classify an input image.

In [3]:
def create_cnn(input_shape, output_classes):
    return keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dense(output_classes, activation="softmax"),
        ]
    )

## Prepare the data
The neural network will be trained on a digit classification dataset called *MNIST*. This code downloads and loads the images together with their true labels. The code also does some preprocessing of the data to make it more suitable for a neural network.

In [4]:
def get_mnist_data():
    # Load the data and split it between train and test sets
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Scale images to the [0, 1] range
    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255

    # Make sure images have shape (28, 28, 1)
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)

    # convert class vectors to binary class matrices
    num_classes = 10
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    return (x_train, y_train), (x_test, y_test)

## Train the network
Finally we will train the network on the data to teach it how to classify a digit. We create a model which expects a 28x28 pixel monocolor image since this is the format the images in the *MNIST* dataset are. We then create an optimizer and calls the `fit()` method to start the training.

In [12]:
batch_size = 128
epochs = 1

In [13]:
# Get the training data
x_train, y_train = get_mnist_data()[0]

In [14]:
# Create a Convolutional Neural Network that
# expects a 28x28 pixel image with 1 color chanel (gray) as input
model = create_cnn((28, 28, 1), 10)

model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size,
          epochs=epochs, validation_split=0.1)

422/422 [==============================] - 42s 98ms/step - loss: 0.2918 - accuracy: 0.9184 - val_loss: 0.0841 - val_accuracy: 0.9777


In [8]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 23.3 MB/s 
     |████████████████████████████████| 168 kB 51.5 MB/s 
     |████████████████████████████████| 182 kB 81.4 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 168 kB 55.1 MB/s 
     |████████████████████████████████| 166 kB 56.8 MB/s 
     |████████████████████████████████| 166 kB 59.6 MB/s 
     |████████████████████████████████| 162 kB 60.8 MB/s 
     |████████████████████████████████| 162 kB 57.9 MB/s 
     |████████████████████████████████| 158 kB 61.4 MB/s 
     |████████████████████████████████| 157 kB 53.4 MB/s 
     |████████████████████████████████| 157 kB 60.1 MB/s 
     |████████████████████████████████| 157 kB 59.5 MB/s 
     |████████████████████████████████| 157 kB 59.5 MB/s 
     |████████████████████████████████| 157 kB 61.1 MB/s 
     |███████████████████████████

In [10]:
# Log in to your W&B account
import wandb
wandb.login()

wandb: Currently logged in as: amir_anissian. Use `wandb login --relogin` to force relogin


True

In [11]:
wandb.init(project="ai-ml-exercise", config={"batch_size": batch_size, "epochs": epochs})

In [16]:
model.fit(x_train, y_train, batch_size=batch_size,
          epochs=epochs, validation_split=0.1,
          callbacks=[wandb.keras.WandbCallback()])
wandb.finish()

422/422 [==============================] - ETA: 0s - loss: 0.0551 - accuracy: 0.9834

wandb: Adding directory to artifact (/content/wandb/run-20221206_125646-2k1firdv/files/model-best)... Done. 0.0s


422/422 [==============================] - 43s 101ms/step - loss: 0.0551 - accuracy: 0.9834 - val_loss: 0.0474 - val_accuracy: 0.9882


accuracy,▁█
epoch,▁▁
loss,█▁
val_accuracy,▁█
val_loss,█▁
accuracy,0.98341
best_epoch,0
best_val_loss,0.04739
epoch,0
loss,0.05511
val_accuracy,0.98817
